# 🤖 Entraînement RandomForest - Classificateur de Triage

**Dataset :** 79 cas équilibrés (20 ROUGE, 19 JAUNE, 20 VERT, 20 GRIS)

**Features :** 776 dimensions
- 768 : Embeddings CamemBERT-bio (symptômes)
- 6 : Constantes vitales (FC, FR, SpO2, TA, Temp)
- 2 : Patient (âge, sexe)

## 1️⃣ Imports

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    f1_score
)



## 2️⃣ Charger le dataset

In [ ]:
# Charger le dataset COMPLET (79 cas)
data_path = Path('../data/triage_dataset_complete.pkl')

with open(data_path, 'rb') as f:
    data = pickle.load(f)

X = data['X']
y = data['y']

print(f"✅ Dataset chargé !")
print(f"   X shape : {X.shape}")
print(f"   y shape : {y.shape}")
print(f"   Features : {X.shape[1]} dimensions")
print(f"   Cas : {X.shape[0]}")

## 3️⃣ Distribution des classes

In [ ]:
from collections import Counter

counts = Counter(y)

print("📊 DISTRIBUTION DES CLASSES")
print("="*60)
for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']:
    count = counts[label]
    pct = (count / len(y)) * 100
    bar = '█' * (count // 2)
    print(f"   {label:6s} : {count:2d} ({pct:5.1f}%) {bar}")
print("="*60)

# Visualisation
plt.figure(figsize=(10, 6))
colors = {'ROUGE': 'red', 'JAUNE': 'yellow', 'VERT': 'green', 'GRIS': 'gray'}
bar_colors = [colors[label] for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']]
values = [counts[label] for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']]

plt.bar(['ROUGE', 'JAUNE', 'VERT', 'GRIS'], values, color=bar_colors, alpha=0.7, edgecolor='black')
plt.title('Distribution des Classes', fontsize=16, fontweight='bold')
plt.ylabel('Nombre de Cas')
plt.xlabel('Niveau de Gravité')
plt.grid(axis='y', alpha=0.3)

for i, v in enumerate(values):
    plt.text(i, v + 0.5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 4️⃣ Split Train/Test

In [ ]:
# Split stratifié (maintient distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25,  # 25% test
    random_state=42,
    stratify=y  # Important : maintient distribution
)

print(f"📊 SPLIT TRAIN/TEST")
print(f"="*60)
print(f"   Train : {len(X_train)} cas ({len(X_train)/len(X)*100:.0f}%)")
print(f"   Test  : {len(X_test)} cas ({len(X_test)/len(X)*100:.0f}%)")
print(f"="*60)

# Vérifier distribution train
print(f"\n   Distribution TRAIN :")
train_counts = Counter(y_train)
for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']:
    print(f"      {label} : {train_counts[label]} cas")

print(f"\n   Distribution TEST :")
test_counts = Counter(y_test)
for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']:
    print(f"      {label} : {test_counts[label]} cas")

## 5️⃣ Entraîner RandomForest

In [ ]:
print("🎓 Entraînement du modèle...\n")

# Créer et entraîner
clf = RandomForestClassifier(
    n_estimators=100,  # 100 arbres
    max_depth=10,      # Profondeur max
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1          # Utiliser tous les CPU
)

clf.fit(X_train, y_train)

print("✅ Modèle entraîné !")
print(f"   Nombre d'arbres : {clf.n_estimators}")
print(f"   Profondeur max : {clf.max_depth}")

## 6️⃣ Prédictions

In [ ]:
# Prédictions sur test
y_pred = clf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"📊 PERFORMANCES SUR TEST SET")
print(f"="*60)
print(f"   Accuracy : {accuracy*100:.1f}%")
print(f"   F1-Score : {f1:.3f}")
print(f"="*60)

## 7️⃣ Rapport de classification

In [ ]:
print("\n📋 RAPPORT DE CLASSIFICATION DÉTAILLÉ")
print("="*60)
print(classification_report(
    y_test, 
    y_pred,
    target_names=['GRIS', 'JAUNE', 'ROUGE', 'VERT'],
    zero_division=0
))

## 8️⃣ Matrice de confusion

In [ ]:
# Calculer matrice
cm = confusion_matrix(y_test, y_pred, labels=['ROUGE', 'JAUNE', 'VERT', 'GRIS'])

# Afficher texte
print("\n🔢 MATRICE DE CONFUSION")
print("="*60)
cm_df = pd.DataFrame(
    cm,
    index=['ROUGE', 'JAUNE', 'VERT', 'GRIS'],
    columns=['ROUGE', 'JAUNE', 'VERT', 'GRIS']
)
print(cm_df)
print("\n(Lignes = Vrai, Colonnes = Prédit)")

# Visualiser
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm, 
    annot=True, 
    fmt='d', 
    cmap='Blues',
    xticklabels=['ROUGE', 'JAUNE', 'VERT', 'GRIS'],
    yticklabels=['ROUGE', 'JAUNE', 'VERT', 'GRIS'],
    cbar_kws={'label': 'Nombre de cas'}
)
plt.title('Matrice de Confusion', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Classe Réelle', fontweight='bold')
plt.xlabel('Classe Prédite', fontweight='bold')
plt.tight_layout()
plt.show()

## 9️⃣ Validation croisée

In [ ]:
# Cross-validation 5-fold
print("🔄 VALIDATION CROISÉE (5-fold)")
print("="*60)

cv_scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')

print(f"   Scores par fold :")
for i, score in enumerate(cv_scores, 1):
    print(f"      Fold {i} : {score*100:.1f}%")

print(f"\n   Moyenne : {cv_scores.mean()*100:.1f}% (± {cv_scores.std()*100:.1f}%)")
print("="*60)

## 🔟 Importance des features

In [ ]:
# Top 20 features les plus importantes
feature_importance = clf.feature_importances_

# Indices des constantes vitales
vitals_indices = {
    768: 'FC (fréquence cardiaque)',
    769: 'FR (fréquence respiratoire)',
    770: 'SpO2 (saturation)',
    771: 'TA systolique',
    772: 'TA diastolique',
    773: 'Température',
    774: 'Âge',
    775: 'Sexe'
}

print("\n🔝 TOP 10 FEATURES LES PLUS IMPORTANTES")
print("="*60)

# Trier par importance
indices = np.argsort(feature_importance)[::-1][:10]

for rank, idx in enumerate(indices, 1):
    importance = feature_importance[idx]
    if idx in vitals_indices:
        name = vitals_indices[idx]
    else:
        name = f"Embedding symptôme #{idx}"
    print(f"   {rank:2d}. {name:30s} : {importance:.4f}")

print("="*60)

## 1️⃣1️⃣ Test sur cas spécifiques

In [ ]:
print("\n🧪 TEST SUR CAS SPÉCIFIQUES")
print("="*60)

# Prendre 5 cas aléatoires du test set
np.random.seed(42)
test_indices = np.random.choice(len(X_test), size=min(5, len(X_test)), replace=False)

for i, idx in enumerate(test_indices, 1):
    x_case = X_test[idx:idx+1]
    y_true = y_test[idx]
    y_pred_case = clf.predict(x_case)[0]
    proba = clf.predict_proba(x_case)[0]
    
    # Probabilités par classe
    proba_dict = dict(zip(clf.classes_, proba))
    
    print(f"\n   Cas #{i}")
    print(f"      Vrai label : {y_true}")
    print(f"      Prédiction : {y_pred_case} {'✅' if y_pred_case == y_true else '❌'}")
    print(f"      Confiance :")
    for label in ['ROUGE', 'JAUNE', 'VERT', 'GRIS']:
        prob = proba_dict.get(label, 0)
        bar = '█' * int(prob * 20)
        print(f"         {label:6s} : {prob*100:5.1f}% {bar}")

print("\n" + "="*60)

## 1️⃣2️⃣ Sauvegarder le modèle

In [ ]:
# Créer dossier models
models_dir = Path('../models')
models_dir.mkdir(exist_ok=True)

# Sauvegarder
model_path = models_dir / 'random_forest_triage.pkl'

with open(model_path, 'wb') as f:
    pickle.dump(clf, f)

print(f"✅ Modèle sauvegardé : {model_path}")
print(f"\n📦 Taille du fichier : {model_path.stat().st_size / 1024:.1f} KB")

## ✅ Résumé & Conclusion

In [ ]:
print("\n" + "="*70)
print("📊 RÉSUMÉ FINAL")
print("="*70)
print(f"\n✅ DATASET")
print(f"   Total : {len(X)} cas")
print(f"   Features : {X.shape[1]} dimensions")
print(f"   Classes : 4 (ROUGE, JAUNE, VERT, GRIS)")
print(f"   Équilibré : Oui")

print(f"\n✅ MODÈLE")
print(f"   Algorithme : RandomForest")
print(f"   Arbres : {clf.n_estimators}")
print(f"   Accuracy : {accuracy*100:.1f}%")
print(f"   F1-Score : {f1:.3f}")

print(f"\n✅ FICHIERS CRÉÉS")
print(f"   Modèle : models/random_forest_triage.pkl")

print(f"\n💡 PROCHAINES ÉTAPES")
print(f"   1. Intégrer dans Streamlit")
print(f"   2. Tester avec nouveaux patients")
print(f"   3. Améliorer avec plus de données")

print("\n" + "="*70)
print("🎉 ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !")
print("="*70)